In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam # - Works

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
# import split-folders
import numpy as np
import cv2
import os


import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

import tensorflow as tf
import tensorflow_addons as tfa


Imported libraries and using transfer learning(pre-trained models)

In [ ]:
mobilenetV2 = MobileNetV2(input_shape=(224, 224, 3),
                          include_top=False,
                          weights='imagenet'
                          )

average_layer = GlobalAveragePooling2D()

model = Sequential([
    mobilenetV2,
    average_layer,
    Dense(256, activation=tf.nn.relu),
    BatchNormalization(),
    Dropout(0.2),
    Dense(2, activation=tf.nn.softmax)
])

In [ ]:
train_path = "../input/fake-image-classification-challenge/data/train"
test_path = "../input/fake-image-classification-challenge/data/test"

In [ ]:
NUM_CLASSES = len(os.listdir(train_path))
BATCH_SIZE = 32
SEED = 2021
HEIGHT,WIDTH = 224,224
NUM_CLASSES

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2, 
    zoom_range = 0.2,
    validation_split=0.2
   )

train_ds = train_datagen.flow_from_directory(
                             train_path,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = True,
                             seed = SEED,
                             subset = "training",
                             color_mode='rgb'
                            )
val_ds = train_datagen.flow_from_directory(
                             train_path,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                             subset = "validation",
                             color_mode='rgb'
                            )

In [ ]:
model.summary()

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()
        
metrics = [
       tfa.metrics.F1Score(num_classes = NUM_CLASSES,average = "macro", name = "f1_score"),
       tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    ]

model.compile(optimizer=Adam(lr=0.001),
              loss=loss, metrics=metrics)

In [ ]:
MONITOR = "f1_score"

def create_callbacks():
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor=MONITOR,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor=MONITOR,
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor=MONITOR,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

In [ ]:
EPOCHS= 20
VERBOSE =1
callbacks = create_callbacks()

history = model.fit(train_ds, 
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = val_ds,
                        verbose=VERBOSE
                       )

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
f1 = history.history['f1_score']
val_f1 = history.history['val_f1_score']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(history.history['val_loss'])) 

plt.figure(figsize = (15,5))
plt.subplot(1, 3,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation  Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation  Accuracy')
    
plt.subplot(1, 3,2)
plt.plot(epochs_range, f1, label='Training F1 score')
plt.plot(epochs_range, val_f1, label='Validation  F1 score')
plt.legend(loc='lower right')
plt.title('Training and Validation  F1 score')
    
plt.subplot(1, 3, 3)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
MODEL_PATH = './best_model.h5'
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
TEST_PATH = "../input/fake-image-classification-challenge/data/"
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
   )
test_ds = test_datagen.flow_from_directory(
                      TEST_PATH,classes=['test'],
                      target_size = (HEIGHT,WIDTH),
                      shuffle = False,
                      batch_size = BATCH_SIZE
                       )

In [ ]:
test_dir = pd.read_csv('../input/fake-image-classification-challenge/data/test.csv')

In [ ]:
pred = model.predict(test_ds)
predictions = [np.argmax(i) for i in pred]
#change 0 to fake and 1 to real of predictions rray

In [ ]:
test_df = pd.DataFrame(test_dir, columns = ["path"])
test_df["label"] = predictions
test_df.to_csv('./submission.csv',index=False)
test_df.head()